In [1]:
from __future__ import print_function
import argparse
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np
import pandas as pd
import os
import pickle
from sklearn import preprocessing
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity

## Read the Data Sets

In [2]:
path = 'Path to the Resampled Time Series'
with open(path + 'final_train_ihm.pickle', 'rb') as handle:
    train = pickle.load(handle)
with open(path +'final_test_ihm.pickle', 'rb') as handle:
    test = pickle.load(handle)
train_lab = np.array(pd.read_csv(path + 'Data\\train\\listfile.csv').iloc[:,1])
test_lab = np.array(pd.read_csv(path + 'Data\\test\\listfile.csv').iloc[:,1])
dis_train = pd.read_csv(path+'Data\\disease_diagnostics_train.csv',index_col=0)
dis_test = pd.read_csv(path+'Data\\disease_diagnostics_test.csv',index_col=0)
dis_train = np.array(dis_train.sum(axis = 1))
dis_test = np.array(dis_test.sum(axis = 1))

## Filter the Train and Test Data Set so new Data Set have at max 10 % missing Values

In [3]:
missing = pd.DataFrame ([train [idx][2].sum() for idx in range(len(train)) ])
temp = missing.sum (axis = 1)
Indices = list(temp[temp<=33].index)
tr_indices = Indices
train = [train [idx] for idx in Indices]
train_lab = train_lab[Indices]
dis_train = dis_train[Indices]
missing = pd.DataFrame ([test [idx][2].sum() for idx in range(len(test)) ])
temp = missing.sum (axis = 1)
Indices = list(temp[temp<=31].index)
te_indices = Indices
test = [test [idx] for idx in Indices]
test_lab = test_lab[Indices]
dis_test = dis_test[Indices]

## Fill the Missing Values by Mean

In [4]:
missing = pd.DataFrame([ train [idx][2].sum() for idx in range(len(train)) ])
missing = [ np.where(missing.iloc[:,column] != 0 ) [0] for column in range(1,len(missing.columns)) ]
train_resampled = [train[idx][1].fillna(train[idx][1].mean()) for idx in range(len(train)) ]
test_resampled = [test[idx][1].fillna(test[idx][1].mean()) for idx in range(len(test)) ]
m_train = np.sum(np.array([train_resampled[idx].isna().sum().sum() for idx in range(len(train_resampled)) ]))
m_test = np.sum(np.array([test_resampled[idx].isna().sum().sum() for idx in range(len(test_resampled)) ]))
print (m_train , m_test)

0 0


## Standardize the Data

In [5]:
cols = train_resampled[0].columns
scalar = MinMaxScaler((-1,1)).fit(pd.concat(train_resampled))
train_resampled = [ pd.DataFrame(scalar.transform (train_resampled[idx])) for idx in range(len(train_resampled)) ]
test_resampled = [ pd.DataFrame(scalar.transform (test_resampled[idx])) for idx in range(len(test_resampled)) ]
for df in train_resampled:
    df.columns = cols
for df in test_resampled:
    df.columns = cols

In [6]:
path = 'Path to the Resampled Time Series'
with open(path + 'final_train_ihm.pickle', 'rb') as handle:
    train = pickle.load(handle)
with open(path +'final_test_ihm.pickle', 'rb') as handle:
    test = pickle.load(handle)
train_lab = np.array(pd.read_csv(path[:-17] + 'Data\\train\\listfile.csv').iloc[:,1])
test_lab = np.array(pd.read_csv(path[:-17] + 'Data\\test\\listfile.csv').iloc[:,1])
dis_train = pd.read_csv(path[:-17]+'Data\\disease_diagnostics_train.csv',index_col=0)
dis_test = pd.read_csv(path[:-17]+'Data\\disease_diagnostics_test.csv',index_col=0)
dis_train = dis_train.iloc[tr_indices, :]
dis_test = dis_test.iloc[te_indices, :]

In [16]:
sim_tr = cosine_similarity(dis_train)
for i in range(len(sim_tr)):
    for j in range (sim_tr.shape[1]):
        if i==j:
            sim_tr [i,j] = 0
sim_tr =  np.triu (sim_tr)
sim_te = cosine_similarity(dis_test)
for i in range(len(sim_te)):
    for j in range (sim_te.shape[1]):
        if i==j:
            sim_te [i,j] = 0
sim_te =  np.triu (sim_te)
sim_tr = [ sim_tr[idx].argsort()[-5:][::-1] for idx in range(len(sim_tr)) ]
sim_te = [ sim_te[idx].argsort()[-5:][::-1] for idx in range(len(sim_te)) ]
train_resampled = [ _concatenate (idx , train_resampled , sim_tr) for idx in range(len(sim_tr)) ]
test_resampled = [ _concatenate (idx , test_resampled , sim_te) for idx in range(len(sim_te)) ]

## Save the Files into Pickle Files

In [20]:
with open('train_resampled.pickle', 'wb') as handle:
    pickle.dump(train_resampled, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('test_resampled.pickle', 'wb') as handle:
    pickle.dump(test_resampled, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('train_lab.pickle', 'wb') as handle:
    pickle.dump(train_lab, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('test_lab.pickle', 'wb') as handle:
    pickle.dump(test_lab, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('dis_train.pickle', 'wb') as handle:
    pickle.dump(dis_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('dis_test.pickle', 'wb') as handle:
    pickle.dump(dis_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Scalar.pickle', 'wb') as handle:
    pickle.dump(scalar, handle, protocol=pickle.HIGHEST_PROTOCOL)